In [3]:
import mysql.connector
import csv
import os
import shutil
import zipfile36 as zipfile
import zipfile as zipfile2
import pyzipper
import paramiko

from datetime import datetime, timedelta
from os import path
from shutil import make_archive

db = mysql.connector.connect(
   host="116.206.196.7",
    port="3309",
   user="reza_cro",
   passwd="Rezacro@123"
)

mycursor = db.cursor()
sql = "SELECT * FROM cro_reports.upload_to_sik where tgl_pelaporan_data=DATE_FORMAT(now()-INTERVAL 1 DAY, '%Y%m%d') "
mycursor.execute(sql)

#Export csv

csv_file = '820012' + datetime.strftime(datetime.now() - timedelta(1), '%Y%m%d') +'SIK05' +'.csv'
csv_file_full = os.path.join("C:", os.sep, "Users", "reza.andreawan", "Documents", "query","Ojk","structure","upload to ojk","upload to sik",csv_file)

with open(csv_file_full,'w',newline="") as f:
    writer = csv.writer(f,delimiter='|')
    for row in mycursor.fetchall():
        writer.writerow(row)

        

#file to zip

csv_file = '820012' + datetime.strftime(datetime.now() - timedelta(1), '%Y%m%d') +'SIK05' +'.csv'
csv_file_zip = '820012' + datetime.strftime(datetime.now() - timedelta(1), '%Y%m%d') +'SIK05' +'.zip'
csv_file_zip_full = os.path.join("C:", os.sep, "Users", "reza.andreawan", "Documents", "query","Ojk","structure","upload to ojk","upload to sik",csv_file_zip)
csv_file_full = os.path.join("C:", os.sep, "Users", "reza.andreawan", "Documents", "query","Ojk","structure","upload to ojk","upload to sik",csv_file)
z1=zipfile2.ZipFile(csv_file_zip_full, mode='w').write(csv_file_full,os.path.basename(csv_file_full))


#to encrypt password
secret_password = b'inipunyauangteman'

with pyzipper.AESZipFile(csv_file_zip_full,
                         'w',
                         compression=pyzipper.ZIP_DEFLATED,
                         encryption=pyzipper.WZ_AES) as zf:
    zf.pwd = secret_password
    zf.write(csv_file_full,os.path.basename(csv_file_full))
    
#send sftp FDC
import paramiko
from datetime import datetime, timedelta
paramiko.util.log_to_file("paramiko.log")

# Open a transport
host = "149.129.250.37"
transport = paramiko.Transport((host))

# Auth    
username,password = "820012","6@Ni$&5266SvR}$s"
transport.connect(None,username,password)

# Go!    
sftp = paramiko.SFTPClient.from_transport(transport)

# Upload
filename= '820012' + datetime.strftime(datetime.now() - timedelta(1), '%Y%m%d') +'SIK05' +'.zip'
filepath = r'/in/'+filename
localpath = r'C:/Users/reza.andreawan/Documents/query/Ojk/structure/upload to ojk/upload to sik/'+ filename
sftp.put(localpath,filepath)

# Close
if sftp: sftp.close()
if transport: transport.close()

20191204
